In [1]:
print("STARTO")
import time
import os
start_time = time.time()

using_gpu = True
if not using_gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = ""

from lib.detector.YOLO.train.trainer import Trainer
from lib.detector.YOLO import create_detector_model_from_trained
from lib.detector.YOLO import create_training_model_from_detector
from lib.detector.YOLO import Yolo

STARTO


Using TensorFlow backend.


In [2]:
from keras.models import load_model
our_ver = load_model('./model_data/v2.h5')

/src/keras/engine/saving.py:269: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
labels = ['bicycle','bus','car','motorbike']
our_yolo = Yolo(labels=labels, yolo_model=our_ver)

In [4]:
import numpy as np
import json
with open('./train_val_config/ua_voc_4.json', 'r') as fp:
    data = json.load(fp)
    train_imgs = data['train']
    valid_imgs = data['val']

In [5]:
import cv2
# "boxes":[], "scores":[]
tests = valid_imgs
gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}

for _dat in valid_imgs:
    key = _dat['filename']
    for obj in _dat['object']:
        gt_boxes[obj['name']][key].append([obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax']])

    _img = cv2.imread(key)
    rgb_img = cv2.cvtColor(_img, cv2.COLOR_BGR2RGB)
#     obj['ymin'], obj['xmin'], obj['ymax'], obj['xmax']
    start_time = time.time()
    results = our_yolo.get_localization(rgb_img, 
                              score_threshold=0.25,
                              class_threshold={
                                  'bicycle': 0.25,
                                  'bus': 0.25,
                                  'car': 0.25,
                                  'motorbike': 0.25
                              })
    end_time = time.time() - start_time    
    for cls_name in results:
        pred_boxes[cls_name][key]['boxes'] = []
        for box in results[cls_name]['boxes']:
            pred_boxes[cls_name][key]['boxes'].append([box[1], box[0], box[3], box[2]])
        pred_boxes[cls_name][key]['scores'] = results[cls_name]['scores']


In [6]:
# gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
# pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}
import json

def myconverter(o):
    if isinstance(o, np.float32):
        return float(o)
    
with open('gt_boxes_v2.json', 'w') as outfile:
    json.dump(gt_boxes, outfile, default=myconverter)
with open('pred_boxes_v2.json', 'w') as outfile:
    json.dump(pred_boxes, outfile, default=myconverter)

In [7]:
!mv gt_boxes_v2.json analysis/gt_boxes_v2.json
!mv pred_boxes_v2.json analysis/pred_boxes_v2.json